In [ ]:
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np

from snap_fit.aruco.aruco_board import ArucoBoardGenerator
from snap_fit.config.aruco.aruco_board_config import ArucoBoardConfig
from snap_fit.puzzle import PuzzleConfig
from snap_fit.puzzle import PuzzleGenerator
from snap_fit.puzzle import PuzzleRasterizer
from snap_fit.puzzle import PuzzleSheetComposer
from snap_fit.puzzle import SheetLayout

## 1. Configure the Puzzle


In [ ]:
# Create puzzle configuration
puzzle_config = PuzzleConfig(
    width=200,  # 200mm wide puzzle
    height=150,  # 150mm tall puzzle
    tiles_x=8,  # 8 columns
    tiles_y=6,  # 6 rows
    seed=42,  # Reproducible
    tab_size=0.2,
    jitter=0.04,
    corner_radius=1.5,
    font_size=3,
)

print(
    f"Puzzle: {puzzle_config.tiles_x}x{puzzle_config.tiles_y} = {puzzle_config.tiles_x * puzzle_config.tiles_y} pieces"
)
print(f"Size: {puzzle_config.width}x{puzzle_config.height} mm")
print(
    f"Piece size: {puzzle_config.piece_width:.1f}x{puzzle_config.piece_height:.1f} mm"
)
print(
    f"Label format: {puzzle_config.letter_digits} letter(s) + {puzzle_config.number_digits} digit(s)"
)

## 2. Generate the Puzzle


In [ ]:
# Generate puzzle
generator = PuzzleGenerator(puzzle_config)
pieces = generator.generate()

print(f"Generated {len(pieces)} pieces")
print(f"First 5 labels: {[p.label for p in pieces[:5]]}")
print(f"Last 5 labels: {[p.label for p in pieces[-5:]]}")

In [ ]:
# Show full puzzle SVG
from IPython.display import SVG
from IPython.display import display

svg = generator.to_svg(include_labels=True)
display(SVG(svg))

## 3. Rasterize the Puzzle


In [ ]:
# Rasterize at high DPI
rasterizer = PuzzleRasterizer(dpi=300)
puzzle_img = rasterizer.rasterize(svg)

print(f"Rasterized image: {puzzle_img.shape}")

plt.figure(figsize=(14, 10))
plt.imshow(puzzle_img[:, :, ::-1])
plt.axis("off")
plt.title("Full Puzzle (Rasterized)")
plt.show()

## 4. Generate ArUco Board

Create an ArUco board image to use as the background for puzzle sheets.


In [ ]:
# Configure ArUco board
aruco_config = ArucoBoardConfig(
    markers_x=7,
    markers_y=5,
    marker_length=100,  # mm
    marker_separation=100,  # mm
)

# Generate ArUco board
aruco_generator = ArucoBoardGenerator(aruco_config)
# aruco_img = aruco_generator.generate_board_image(dpi=300)
aruco_img = aruco_generator.generate_image()

print(f"ArUco board image: {aruco_img.shape}")

plt.figure(figsize=(10, 8))
plt.imshow(aruco_img, cmap="gray")
plt.axis("off")
plt.title("ArUco Board")
plt.show()

## 5. Configure Sheet Layout


In [ ]:
# A4 sheet layout
sheet_layout = SheetLayout(
    sheet_width=297,  # A4 width in mm
    sheet_height=210,  # A4 height in mm
    margin=50,  # Space for ArUco markers
    piece_spacing=20,  # Gap between pieces
    dpi=300,
)

pieces_per_row, pieces_per_col = sheet_layout.pieces_per_sheet(
    puzzle_config.piece_width, puzzle_config.piece_height
)

print(f"Sheet: {sheet_layout.sheet_width}x{sheet_layout.sheet_height} mm")
print(f"Usable area: {sheet_layout.usable_width}x{sheet_layout.usable_height} mm")
print(
    "Pieces per sheet:"
    f" {pieces_per_row}x{pieces_per_col} = {pieces_per_row * pieces_per_col}"
)

total_pieces = puzzle_config.tiles_x * puzzle_config.tiles_y
sheets_needed = -(
    -total_pieces // (pieces_per_row * pieces_per_col)
)  # Ceiling division
print(f"Total pieces: {total_pieces}")
print(f"Sheets needed: {sheets_needed}")

## 6. Compose Sheets with Pieces


In [ ]:
# Convert ArUco image to BGR if needed
if len(aruco_img.shape) == 2:
    aruco_img_bgr = cv2.cvtColor(aruco_img, cv2.COLOR_GRAY2BGR)
else:
    aruco_img_bgr = aruco_img

# Create sheet composer
composer = PuzzleSheetComposer(sheet_layout, aruco_img_bgr)

# Generate all sheets
sheets = composer.generate_all_sheets(generator)

print(f"Generated {len(sheets)} sheets")

In [ ]:
# Display the sheets
for i, sheet in enumerate(sheets):
    plt.figure(figsize=(14, 10))
    plt.imshow(sheet[:, :, ::-1])
    plt.axis("off")
    plt.title(f"Sheet {i + 1}/{len(sheets)}")
    plt.show()
    break  # Show only the first sheet for brevity

## 7. Save Output


In [ ]:
# Create output directory
output_dir = Path("output")
output_dir.mkdir(exist_ok=True)

# Save full puzzle
puzzle_svg_path = output_dir / "puzzle_full.svg"
puzzle_svg_path.write_text(svg)
print(f"Saved: {puzzle_svg_path}")

puzzle_img_path = output_dir / "puzzle_full.png"
cv2.imwrite(str(puzzle_img_path), puzzle_img)
print(f"Saved: {puzzle_img_path}")

# Save sheets
sheet_paths = composer.save_sheets(sheets, str(output_dir), prefix="puzzle_sheet")
print(f"Saved {len(sheet_paths)} sheets")

## 8. Save Individual Pieces (Optional)

Save each piece as a separate image for testing.


In [ ]:
# Save first few pieces as examples
pieces_dir = output_dir / "pieces"
pieces_dir.mkdir(exist_ok=True)

for piece in pieces[:6]:
    piece_svg = generator.piece_to_svg(piece.row, piece.col, include_label=True)
    piece_img = rasterizer.rasterize(piece_svg)

    piece_path = pieces_dir / f"piece_{piece.label}.png"
    cv2.imwrite(str(piece_path), piece_img)
    print(f"Saved: {piece_path}")

In [ ]:
print("\n" + "=" * 50)
print("Puzzle generation complete!")
print("=" * 50)
print(f"\nOutput directory: {output_dir.absolute()}")
print("\nFiles:")
for f in sorted(output_dir.rglob("*")):
    if f.is_file():
        print(f"  {f.relative_to(output_dir)}")